In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
df = pd.read_csv('academy_awards.csv',encoding='ISO-8859-1')

In [3]:
print(df.size)
df.columns

111507


Index(['Year', 'Category', 'Nominee', 'Additional Info', 'Won?', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'],
      dtype='object')

In [4]:
df.iloc[:,:5].head(6)

,Year,Category,Nominee,Additional Info,Won?
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO
2,2010 (83rd),Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO
3,2010 (83rd),Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES
4,2010 (83rd),Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO
5,2010 (83rd),Actor -- Supporting Role,Christian Bale,The Fighter {'Dicky Eklund'},YES


In [5]:
df.iloc[:,5].value_counts('NaN')

*                                                                                                               0.636364
 error-prone measurements on sets. [Digital Imaging Technology]"                                                0.090909
 D.B. "Don" Keele and Mark E. Engebretson has resulted in the over 20-year dominance of constant-directivity    0.090909
 discoverer of stars                                                                                            0.090909
 resilience                                                                                                     0.090909
Name: Unnamed: 5, dtype: float64

In [6]:
df.iloc[:,5].value_counts()

*                                                                                                               7
 error-prone measurements on sets. [Digital Imaging Technology]"                                                1
 D.B. "Don" Keele and Mark E. Engebretson has resulted in the over 20-year dominance of constant-directivity    1
 discoverer of stars                                                                                            1
 resilience                                                                                                     1
Name: Unnamed: 5, dtype: int64

In [7]:
#Converting the Year column to int64 type after filtering out string exclosed by brackets.

df['Year'] = df["Year"].str[:4].astype(np.int64)

In [8]:
#Selecting dates later than 2000
later_than_2000  = df[df['Year']>2000]

In [9]:
#Looking up list of award categories and selecting the first four(most hyped) awards
print(later_than_2000['Category'].unique())
award_categories = later_than_2000['Category'].unique()[:4].astype(np.str)
award_categories

['Actor -- Leading Role' 'Actor -- Supporting Role'
 'Actress -- Leading Role' 'Actress -- Supporting Role'
 'Animated Feature Film' 'Art Direction' 'Cinematography' 'Costume Design'
 'Directing' 'Documentary (Feature)' 'Documentary (Short Subject)'
 'Film Editing' 'Foreign Language Film' 'Makeup' 'Music (Scoring)'
 'Music (Song)' 'Best Picture' 'Short Film (Animated)'
 'Short Film (Live Action)' 'Sound' 'Sound Editing' 'Visual Effects'
 'Writing' 'Honorary Award' 'Irving G. Thalberg Memorial Award'
 'Scientific and Technical (Scientific and Engineering Award)'
 'Scientific and Technical (Technical Achievement Award)'
 'Scientific and Technical (Bonner Medal)'
 'Jean Hersholt Humanitarian Award'
 'Scientific and Technical (Gordon E. Sawyer Award)'
 'Scientific and Technical (Academy Award of Merit)'
 'Scientific and Technical (Special Awards)']


array(['Actor -- Leading Role', 'Actor -- Supporting Role',
       'Actress -- Leading Role', 'Actress -- Supporting Role'], 
      dtype='<U26')

In [10]:
#Selecting all rows from the dataframe that pertain to items in the award categories list.

nominations = later_than_2000[later_than_2000['Category'].isin(award_categories)]

In [11]:
#Selecting first five columns

nominations = nominations.iloc[:,:5]
#Renaming the Won? column
nominations.rename(columns={'Won?':'Won'},inplace=True)
#Replacing the values in Won column with boolean values
nominations.replace({"YES":1,"NO":0},inplace=True)

final_nominations = nominations
final_nominations.head(5)

,Year,Category,Nominee,Additional Info,Won
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},0
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},0
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},0
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},1
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},0


In [12]:
#Splitting the Additional Info column into movie names and charachters using vectorized string operations

#Splitting on last }
a = final_nominations['Additional Info'].str.rstrip('}')
#Splitting on first {
a = a.str.split('{')
print(type(a))
a.head(2)

<class 'pandas.core.series.Series'>


0               [Biutiful , 'Uxbal']
1    [True Grit , 'Rooster Cogburn']
Name: Additional Info, dtype: object

In [13]:
#Selecting the first and second element of each row.

movie_names = a.str[0]
charachters = a.str[1]

In [14]:
#Assigning the pd.series to a new column in the Dataframe
final_nominations['Movies'] = movie_names
final_nominations['Charachter'] = charachters

#Dropping the old Additional info column
final_nominations.drop('Additional Info',axis=1,inplace=True)

In [16]:
#Exporting the data to a SQL database

#Creating a new database file
#!rm nominations.db
conn = sqlite3.connect('nominations.db')

final_nominations.to_sql('nominations',conn,index=False) #Not writing the dataframe index as a column

In [17]:
f = conn.execute("pragma table_info(nominations)")
f.fetchall()

[(0, 'id', 'integer', 0, None, 1),
 (1, 'category', 'text', 0, None, 0),
 (2, 'nominee', 'text', 0, None, 0),
 (3, 'movie', 'text', 0, None, 0),
 (4, 'charachter', 'text', 0, None, 0),
 (5, 'Won', 'integer', 0, None, 0),
 (6, 'ceremony_id', 'integer', 0, None, 0)]

In [18]:
f = conn.execute("select * from nominations limit 5")
f.fetchall()

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful ', "'Uxbal'", 0, 1),
 (2,
  'Actor -- Leading Role',
  'Jeff Bridges',
  'True Grit ',
  "'Rooster Cogburn'",
  0,
  1),
 (3,
  'Actor -- Leading Role',
  'Jesse Eisenberg',
  'The Social Network ',
  "'Mark Zuckerberg'",
  0,
  1),
 (4,
  'Actor -- Leading Role',
  'Colin Firth',
  "The King's Speech ",
  "'King George VI'",
  1,
  1),
 (5,
  'Actor -- Leading Role',
  'James Franco',
  '127 Hours ',
  "'Aron Ralston'",
  0,
  1)]